In [1]:
import pandas as pd
import bs4
import csv
import numpy as np
import matplotlib.pyplot as plt      
import sklearn
from sklearn import preprocessing, linear_model, model_selection
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn import linear_model, metrics, preprocessing
from sklearn.metrics import r2_score 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV ,train_test_split
from sklearn.metrics import  mean_absolute_error
import seaborn as sns
from scipy.stats import boxcox
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from ipynb.fs.full.set_up import data_prep


In [2]:
def tune_model(X_train_scaled, y_train):
#     params = {'max_depth': [5, 10, 25], 'n_estimators': [100, 150, 200], 'min_samples_split': [2, 4, 6] }
#     grid_search_cv = GridSearchCV(RandomForestRegressor(), params, n_jobs=-1, verbose=1, cv=7)
#     grid_search_cv.fit(X_train_scaled, y_train)
#     best_params = grid_search_cv.best_params_
#     best_score = grid_search_cv.best_score_
#     print(grid_search_cv.best_estimator_)

    params = {'bootstrap': [True, False],
              'max_depth': [5, 10, 15],
              'max_features': ['auto', 'sqrt'],
              'min_samples_leaf': [1, 2, 4],
              'n_estimators': [100, 150, 200],
              'min_samples_split': [2, 5, 10],
             }
    
    randomized_search_cv = RandomizedSearchCV(RandomForestRegressor(), params, n_jobs=-1, verbose=1, cv=5)
    randomized_search_cv.fit(X_train_scaled, y_train)
    best_params = randomized_search_cv.best_params_
    best_score = randomized_search_cv.best_score_
    
    
    cv_results = randomized_search_cv.cv_results_      
    df = pd.DataFrame(cv_results)
    
    return best_params, best_score , df


def return_best_model(X_train_scaled, y_train, best_params):
    reg = RandomForestRegressor(**best_params)
    reg.fit(X_train_scaled, y_train)
    y_pred = reg.predict(X_test_scaled)
    return reg, y_pred 

 
def evaluate_model(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mae, r2


def find_heavy_tail(df):
    cols = list(df.columns)
    heavy_tails = []
    for col in cols:
        values = df[col].values
        if np.any(values <= 0):
            continue
        skewness = pd.Series(values).skew()
        if abs(skewness) > 0.5:
            transformed, lambda_value = boxcox(values + 1)
            df[col] = transformed
    return df 

In [3]:

X_train_scaled,  X_test_scaled  ,y_train, y_test , X_train, X_test = data_prep()


#3. Tune Model
best_params, best_score ,df_2 = tune_model(X_train_scaled, y_train)

#4. Test the model
model, y_pred = return_best_model(X_train_scaled, y_train, best_params)

#5.evaluate the model
mae, r2 = evaluate_model(y_test, y_pred)

print("Best Parameters:", best_params)
print("Best Score:", best_score)
print("MAE:", mae) 
print("R2:", r2)
# X_train.hist()
# plt.figure(figsize=(15,8))
# sns.heatmap(X_train.corr(), annot =True ,cmap ='YlGnBu')

Prep
(58437, 24)
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters: {'n_estimators': 150, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 15, 'bootstrap': False}
Best Score: 0.7606677038532844
MAE: 328685.50060887827
R2: 0.7770866555104357


In [10]:
def recommend_affordable_apartments(df, model):
    y = df['Price']
    X = df.drop('Price', axis=1)
    
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Predict prices using the trained model
    y_pred = model.predict(X_scaled)
    y_pred = y_pred  # bias * 1.22
    
    df["PREDICTED_PRICE"] = y_pred
    df["PREDICTED_PRICE"] = df["PREDICTED_PRICE"].astype(int)
    df['difference'] =  df["Price"] -  df["PREDICTED_PRICE"] 
    
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    print(f'r2_score: {r2} , mae: {mae}')
    return df.sort_values(by="difference")


yad2_df = data_prep(valid = True)
yad_2_results = recommend_affordable_apartments(yad2_df,model)
yad_2_results

Prep
(429, 15)
r2_score: -2.094169673092057 , mae: 3246879.1793759177


,Rooms,Floor,Size,Price,Build_year,Floors,Long,Lat,Year,Distance_sea,Train,Age,PREDICTED_PRICE,difference
363,3,2,50,1700000,1950,3,180116,662276,2023,2873,444,73,3296090,-1596090
26,3,1,60,1950000,1960,1,180105,662281,2023,2861,428,63,3297849,-1347849
231,3,1,60,1980000,1960,1,180116,662276,2023,2873,444,63,3076639,-1096639
157,4,2,70,2690000,1949,3,179616,662533,2023,2318,312,74,3001569,-311569
317,3,1,70,2090000,2023,4,176678,660360,2023,221,5207,0,2326238,-236238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,4,5,195,11250000,1950,5,178210,663850,2023,566,3028,73,1619128,9630872
174,4,12,126,11500000,1993,28,178106,663335,2023,631,2624,30,1814278,9685722
148,5,2,160,11500000,1991,5,178778,664136,2023,1014,2174,32,1603394,9896606
88,4,7,160,11600000,1965,7,179636,665742,2023,1318,1518,58,1567670,10032330
